In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# ==============================
# 1️⃣ Load Dataset
# ==============================

df = pd.read_csv("../data/raw/fake_real_job_postings_3000x25.csv")

# Combine text fields if needed
df["text"] = df["text"].fillna("")

X_text = df["text"]
y = df["label"]

# ==============================
# 2️⃣ Load Tokenizer (Used in Bi-LSTM training)
# ==============================

with open("../models/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

MAX_LEN = 500  # same used during Bi-LSTM training

# ==============================
# 3️⃣ Load Trained Bi-LSTM Model
# ==============================

bilstm_model = load_model("../models/bilstm_model.h5")

# Create feature extractor (remove last dense layer)
from tensorflow.keras.models import Model

feature_extractor = Model(
    inputs=bilstm_model.input,
    outputs=bilstm_model.layers[-2].output  # 256-d layer
)

# ==============================
# 4️⃣ Text Preprocessing
# ==============================

sequences = tokenizer.texts_to_sequences(X_text)
padded = pad_sequences(sequences, maxlen=MAX_LEN)

# ==============================
# 5️⃣ Extract Bi-LSTM Embeddings
# ==============================

bilstm_features = feature_extractor.predict(padded)

# Shape should be (samples, 256)
print("BiLSTM Feature Shape:", bilstm_features.shape)

# ==============================
# 6️⃣ Auxiliary Feature Extraction (TEXT ONLY)
# ==============================

suspicious_keywords = [
    "urgent", "quick money", "no experience",
    "earn daily", "limited offer", "whatsapp",
    "upfront payment"
]

def extract_aux_features(text):
    text_lower = text.lower()
    
    word_count = len(text.split())
    char_count = len(text)
    
    suspicious_count = sum(text_lower.count(word) for word in suspicious_keywords)
    
    capital_ratio = sum(1 for c in text if c.isupper()) / (len(text) + 1)
    
    exclamation_count = text.count("!")
    
    url_count = len(re.findall(r"http[s]?://", text))
    
    email_count = len(re.findall(r"\S+@\S+", text))
    
    phone_count = len(re.findall(r"\b\d{10}\b", text))
    
    return [
        word_count,
        char_count,
        suspicious_count,
        capital_ratio,
        exclamation_count,
        url_count,
        email_count,
        phone_count
    ]

aux_features = np.array([extract_aux_features(t) for t in X_text])

print("Aux Feature Shape:", aux_features.shape)

# ==============================
# 7️⃣ Scale Auxiliary Features
# ==============================

scaler = StandardScaler()
aux_scaled = scaler.fit_transform(aux_features)

# Save scaler
joblib.dump(scaler, "aux_scaler.pkl")

# ==============================
# 8️⃣ Concatenate Features
# ==============================

X_final = np.concatenate([bilstm_features, aux_scaled], axis=1)

print("Final Feature Shape:", X_final.shape)  # should be (samples, 264)

# ==============================
# 9️⃣ Train-Test Split
# ==============================

X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)

# ==============================
# 🔟 Train MLP
# ==============================

mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='relu',
    solver='adam',
    max_iter=50,
    random_state=42
)

mlp.fit(X_train, y_train)

# ==============================
# 1️⃣1️⃣ Evaluate
# ==============================

y_pred = mlp.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# ==============================
# 1️⃣2️⃣ Save MLP
# ==============================

joblib.dump(mlp, "mlp_model.pkl")

print("Training Completed Successfully.")


KeyError: 'text'

In [2]:
# ============================================================
# SMART FAKE JOB DETECTION - CLEAN RETRAINING PIPELINE
# ============================================================

import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
import re
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

print("Libraries loaded ✅")

# ============================================================
# 1️⃣ LOAD DATASET
# ============================================================

df = pd.read_csv("../data/raw/fake_real_job_postings_3000x25.csv")
print("Dataset loaded:", len(df))

text_columns = ["job_title", "job_description", "requirements", "benefits"]
df[text_columns] = df[text_columns].fillna("")

df["merged_text"] = (
    df["job_title"] + " " +
    df["job_description"] + " " +
    df["requirements"] + " " +
    df["benefits"]
)

# ============================================================
# 2️⃣ CLEAN TEXT
# ============================================================

def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

df["clean_text"] = df["merged_text"].apply(clean_text)

print("Text cleaned ✅")

# ============================================================
# 3️⃣ LOAD TOKENIZER
# ============================================================

with open("../models/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

MAX_LEN = 300

sequences = tokenizer.texts_to_sequences(df["clean_text"])
X_text = tf.keras.preprocessing.sequence.pad_sequences(
    sequences,
    maxlen=MAX_LEN,
    padding="post",
    truncating="post"
)

print("Text tokenized and padded:", X_text.shape)

# ============================================================
# 4️⃣ LOAD BILSTM + FEATURE EXTRACTOR
# ============================================================

bilstm_model = tf.keras.models.load_model("../models/bilstm_model.keras")

feature_extractor = tf.keras.Model(
    inputs=bilstm_model.input,
    outputs=bilstm_model.get_layer("bilstm_layer").output
)

X_bilstm = feature_extractor.predict(X_text, batch_size=32)

print("Bi-LSTM features extracted:", X_bilstm.shape)

# ============================================================
# 5️⃣ TEXT-DERIVED AUXILIARY FEATURES
# ============================================================

suspicious_keywords = [
    "urgent", "quick money", "no experience",
    "earn daily", "limited offer",
    "whatsapp", "upfront payment"
]

def extract_aux_features(text):
    text_lower = text.lower()

    word_count = len(text.split())
    char_count = len(text)

    suspicious_count = sum(text_lower.count(word) for word in suspicious_keywords)

    capital_ratio = sum(1 for c in text if c.isupper()) / (len(text) + 1)

    exclamation_count = text.count("!")

    url_count = len(re.findall(r"http[s]?://", text))
    email_count = len(re.findall(r"\S+@\S+", text))
    phone_count = len(re.findall(r"\b\d{10}\b", text))

    return [
        word_count,
        char_count,
        suspicious_count,
        capital_ratio,
        exclamation_count,
        url_count,
        email_count,
        phone_count
    ]

X_aux = np.array([extract_aux_features(t) for t in df["merged_text"]])

print("Auxiliary features extracted:", X_aux.shape)

# ============================================================
# 6️⃣ SCALE AUX FEATURES
# ============================================================

scaler = StandardScaler()
X_aux_scaled = scaler.fit_transform(X_aux)

joblib.dump(scaler, "../models/aux_scaler.pkl")

print("Auxiliary features scaled & scaler saved ")

# ============================================================
# 7️⃣ CONCATENATE FEATURES
# ============================================================

X_final = np.concatenate([X_bilstm, X_aux_scaled], axis=1)

print("Final feature vector shape:", X_final.shape)
# Expected: (?, 256 + 8 = 264)

# ============================================================
# 8️⃣ LABELS
# ============================================================

y = df["is_fake"].values

# ============================================================
# 9️⃣ TRAIN / VAL / TEST SPLIT
# ============================================================

X_train, X_temp, y_train, y_temp = train_test_split(
    X_final, y,
    test_size=0.30,
    random_state=42,
    stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)

print("Data split complete")

# ============================================================
# 🔟 MLP MODEL
# ============================================================

mlp_model = Sequential([
    Dense(128, activation="relu", input_shape=(X_final.shape[1],)),
    Dropout(0.4),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

mlp_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

mlp_model.summary()

# ============================================================
# 1️⃣1️⃣ TRAIN
# ============================================================

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history = mlp_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop]
)

# ============================================================
# 1️⃣2️⃣ EVALUATE
# ============================================================

test_loss, test_accuracy = mlp_model.evaluate(X_test, y_test)

print("Final Test Accuracy:", test_accuracy)
print("Final Test Loss:", test_loss)

# ============================================================
# 1️⃣3️⃣ SAVE MODEL
# ============================================================

mlp_model.save("../models/mlp_model_clean.keras")

print("Clean MLP model saved successfully ")


Libraries loaded ✅
Dataset loaded: 3000
Text cleaned ✅
Text tokenized and padded: (3000, 300)


c:\Users\heman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


94/94 ━━━━━━━━━━━━━━━━━━━━ 9s 95ms/step
Bi-LSTM features extracted: (3000, 256)
Auxiliary features extracted: (3000, 8)
Auxiliary features scaled & scaler saved 
Final feature vector shape: (3000, 264)
Data split complete


c:\Users\heman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        33,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 42,241 (165.00 KB)

 Trainable params: 42,241 (165.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9243 - loss: 0.1596 - val_accuracy: 1.0000 - val_loss: 4.5662e-05
Epoch 2/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 4.5838e-04 - val_accuracy: 1.0000 - val_loss: 1.4669e-05
Epoch 3/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 3.9921e-04 - val_accuracy: 1.0000 - val_loss: 6.2658e-06
Epoch 4/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 9.3711e-05 - val_accuracy: 1.0000 - val_loss: 2.2158e-06
Epoch 5/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 9.8351e-05 - val_accuracy: 1.0000 - val_loss: 1.1958e-06
Epoch 6/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 7.1755e-05 - val_accuracy: 1.0000 - val_loss: 6.5755e-07
Epoch 7/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 3.7007e-05 - val_accuracy: 1.0000 - val_loss: 4.4205e-07
Epoch 8/20
66/66 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0

In [ ]:
# ============================================================
# SMART FAKE JOB DETECTION - MODEL TESTING SCRIPT
# ============================================================

import tensorflow as tf
import numpy as np
import pickle
import re
import joblib

# ============================================================
# 1️⃣ LOAD MODELS
# ============================================================

print("Loading models...")

bilstm_model = tf.keras.models.load_model("../models/bilstm_model.keras")
mlp_model = tf.keras.models.load_model("../models/mlp_model_clean.keras")

with open("../models/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

scaler = joblib.load("../models/aux_scaler.pkl")

feature_extractor = tf.keras.Model(
    inputs=bilstm_model.input,
    outputs=bilstm_model.get_layer("bilstm_layer").output
)

print("Models loaded successfully ")

# ============================================================
# 2️⃣ TEXT CLEANING FUNCTION
# ============================================================

def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

# ============================================================
# 3️⃣ AUX FEATURE FUNCTION
# ============================================================

suspicious_keywords = [
    "urgent", "quick money", "no experience",
    "earn daily", "limited offer",
    "whatsapp", "upfront payment"
]

def extract_aux_features(text):
    text_lower = text.lower()

    word_count = len(text.split())
    char_count = len(text)

    suspicious_count = sum(text_lower.count(word) for word in suspicious_keywords)

    capital_ratio = sum(1 for c in text if c.isupper()) / (len(text) + 1)

    exclamation_count = text.count("!")

    url_count = len(re.findall(r"http[s]?://", text))
    email_count = len(re.findall(r"\S+@\S+", text))
    phone_count = len(re.findall(r"\b\d{10}\b", text))

    return np.array([[
        word_count,
        char_count,
        suspicious_count,
        capital_ratio,
        exclamation_count,
        url_count,
        email_count,
        phone_count
    ]])

# ============================================================
# 4️⃣ PREDICTION FUNCTION
# ============================================================

MAX_LEN = 300

def predict_job_post(text):

    cleaned = clean_text(text)

    sequence = tokenizer.texts_to_sequences([cleaned])
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequence,
        maxlen=MAX_LEN,
        padding="post",
        truncating="post"
    )

    # Bi-LSTM embedding
    bilstm_features = feature_extractor.predict(padded)

    # Auxiliary features
    aux = extract_aux_features(text)
    aux_scaled = scaler.transform(aux)

    # Concatenate
    final_vector = np.concatenate([bilstm_features, aux_scaled], axis=1)

    # Predict
    probability = mlp_model.predict(final_vector)[0][0]

    label = "FAKE" if probability > 0.5 else "REAL"

    print("\nPrediction:", label)
    print("Fake Probability:", round(float(probability), 4))
    print("Real Probability:", round(float(1 - probability), 4))


# ============================================================
# 5️⃣ TEST EXAMPLES
# ============================================================

# # Example 1 — Likely Fake
# fake_example = """
# URGENT HIRING!!! Earn 5000$ weekly from home.
# No experience required. Limited offer.
# Contact via WhatsApp immediately.
# Upfront payment required to secure your position.
# """

# predict_job_post(fake_example)


# # Example 2 — Likely Real
# real_example = """
# We are hiring a Software Engineer with 3+ years of experience in Python and Java.
# Responsibilities include backend API development and database optimization.
# Bachelor's degree required. Competitive salary and health benefits provided.
# """

# predict_job_post(real_example)


Loading models...


c:\Users\heman\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 11 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Models loaded successfully 
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 512ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step

Prediction: REAL
Fake Probability: 0.0907
Real Probability: 0.9093


In [6]:
real_example = """HR Priyanka

5,721 followers

3d

X

+ Follow

#Urgent Hiring Alert - #Immediate Joiners Needed!

#Tech_cloud urgently #Hiring for multiple roles at a reputed American tech company. If you're looking to start your career switch to a #Remote role, this is your chance!

or

Last Date:- 17/01/2026

We welcome both #Freshers and #ExperiencedProfessionals.

Open positions- Full Stack #Developer, #Android Developer, #React Native Developer, #Web Developer, #Backend Developer, #Frontend Developer, UI/UX #Designer, #Graphic Designer, Data #Analyst, #Data Entry

#Experience: 0-4 years

#Working hours: Flexible

#Income: 12k - 95k / Monthly (Based on Interview Performance)

#Location: Remote

Work schedule: 5 days a week Training will be provided for #Freshers.

Note: Please respond only to this post if you're a #Freshers."""

predict_job_post(real_example)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step

Prediction: FAKE
Fake Probability: 0.9646
Real Probability: 0.0354
